In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import os
import aurora_data_api
from dateutil import parser

In [34]:
nameMap = {
    'MEGAN': "Megan Schacht-bonneau",
    'Megan': "Megan Schacht-bonneau",
    "JORDYS": "Jordys Placido",
    "Jordys": "Jordys Placido",
    "NIKKY": "Nikky Voeuth",
    "Nikky": "Nikky Voeuth"
}
resultsMap = {
    'PASSED': 'Pass',
    'FAILED': 'Fail',
    'PASS': 'Pass',
    'FAIL': 'Fail'
}

def getDate(inp):
    dateVal = re.findall('[0-9][0-9]?/..?/2?0?21', inp)[0]
    formatted = parser.parse(dateVal).strftime('%Y-%m-%d')
    return formatted

# Assume that 
def getFailureType(inp):
    if inp[0] == 'W':
        return inp[:6]
    else:
        return 'none'
    
def getBatchNumber(inp):
    mostRecentBatch = inp.iloc[0]
    for i in range(len(inp)):
        val = inp.iloc[i]
        if type(val) == str:
            mostRecentBatch = val.split(' ')[1]
        inp.iloc[i] = mostRecentBatch
    return inp

def getResults(inp):
    return resultsMap.get(inp, inp)

def getTester(inp):
    return nameMap.get(inp, inp)

# Execute a SQL query: helper method to be used by insertEntry
def execute(q):
    try:
        db = aurora_data_api.connect(aurora_cluster_arn = r_ARN, secret_arn=s_ARN, database=dbName)
        c = db.cursor()
        c.execute(q) # Execute the query
        db.commit()
        return c.fetchall() # Display the output of said query
    except Exception as e: # Error occurred during execute
        print('Error with execute:', e)
    
        return (float('inf'), e)
    
def insertEntry(sn, bn, date, tst, ft, fail, msg, dt, fw):
    try:
        q = ''' Insert into QAResults values ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')''' % (sn, bn, date, tst, ft, fail, msg, dt, fw)
        return execute(q)
    except Exception as e:
        print('serialNum', sn)
        print('Issue with insert entry:', e)
    
# Read the path in--get a table formatted to more easily insert entries
def readPath(path, deviceType):
    df = pd.read_excel(path)
    df = df.dropna(subset = ['Unnamed: 1'])
    df = df[df['Unnamed: 0'] != 'BATCH']
    df = df.drop(columns = ['Unnamed: 5'])
    df.columns = ['BatchNumber', 'SerialNumber', 'FunctionalTest', 'LogMessage', 'Tester']
    df['BatchNumber'] = getBatchNumber(df['BatchNumber'])
    df = df[df['FunctionalTest'] != 'RESULT']
    df['FunctionalTest'] = df['FunctionalTest'].apply(getResults)
    df['Tester'] = df['Tester'].apply(getTester)
#     display(df)
    df['Date'] = df['LogMessage'].apply(getDate)
    df['FailureType'] = df['LogMessage'].apply(getFailureType)
    df['DeviceType'] = deviceType
    
    return df

paths = []
directories = [d for d in os.listdir('.') if d[-1] == '1']
allFiles = []
for d in directories:
    currPaths = os.listdir('./' + d)
    allFiles = allFiles + [d + "/" + p for p in currPaths if '$' not in p]
display(allFiles)

['April 2021/Catalina QC Test 04.11 04.17 2021.xlsx',
 'April 2021/Catalina QC Test 04.18 04.24 2021.xlsx',
 'April 2021/Catalina QC Test 04.04 04.10 2021.xlsx',
 'April 2021/Catalina QC Test 04.25 05.01 2021.xlsx',
 'February 2021/Catalina QC Test 01.31 02.06 2021.xlsx',
 'February 2021/Catalina QC Test 02.14 02.20 2021.xlsx',
 'February 2021/Catalina QC Test 02.21 02.27 2021.xlsx',
 'February 2021/Catalina QC Test 02.07 02.13 2021.xlsx',
 'June 2021/Catalina QC Test 06.06 to 06.12 2021.xlsx',
 'June 2021/Catalina QC Test 06.20 to 06.26 2021.xlsx',
 'June 2021/Catalina QC Test 06.27 to 07.03 2021.xlsx',
 'June 2021/Catalina QC Test 06.013 to 06.19 2021.xlsx',
 'July 2021/Catalina QC Test 07.18 to 07.24 2021.xlsx',
 'July 2021/Catalina QC Test 07.11 to 07.17 2021.xlsx',
 'July 2021/Catalina QC Test 07.04 to 07.10 2021.xlsx',
 'January 2021/Catalina QC Test 01.03 01.09 2021 Restored January 08, 2021 at 08.06.49.108 AM PST.xlsx',
 'January 2021/Catalina QC Test 01.10 01.16 2021.xlsx',
 '

In [26]:
port = 3306
user = 'admin'
password = 'password'

host = 'dashboarddb.cluster-cyvmmsfjcmrl.us-east-1.rds.amazonaws.com'
dbName = 'db'
r_ARN = 'arn:aws:rds:us-east-1:319481862133:cluster:dashboarddb'
s_ARN = 'arn:aws:secretsmanager:us-east-1:319481862133:secret:dashboardDBKey-6XBatB'
def insertTable(res):
    for i in range(len(res)):
        row = res.iloc[i]
        bn = row['BatchNumber']
        sn = row['SerialNumber']
        tst = row['Tester']
        msg = row['LogMessage']
        ft = row['FunctionalTest']
        fail = row['FailureType']
        date = row['Date']
        dt = row['DeviceType']
        fw = '10.42'
        insertEntry(sn, bn, date, tst, ft, fail, msg, dt, fw)
    print('success!')

In [36]:
device = 'Catalina' # Get the type of device

# April Entries are all inserted
res1 = readPath(allFiles[0], 'Catalina') # Read the information from that path
res1.head()
res2 = readPath(allFiles[1], device)
res2.head()
res3 = readPath(allFiles[2], device)
res3.head()


# Tables for Catalina: March
res4 = readPath(allFiles[-5], device)
display(res4.head())
res5 = readPath(allFiles[-4], device)
display(res5.head())
res6 = readPath(allFiles[-3], device)
display(res6.head())
res7 = readPath(allFiles[-2], device) # Missing some dates--pretend they're all completed on 03/23/2021
display(res7.head())
res8 = readPath(allFiles[-1], device)
display(res8.head())

''' Still need to deal with Jan, Feb, Jun, Jul'''

,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,UK20200928-A10D,G8T1-GJ00-0301-1LS2,Pass,WHW-70 (SERIAL STICKER IN WRONG SPOT)//COMPLET...,Megan Schacht-bonneau,2021-03-30,WHW-70,Catalina
2,UK20200928-A10D,G8T1-GJ00-0301-1LUW,Pass,WHW-70//COMPLETED ON 03/30/2021,Megan Schacht-bonneau,2021-03-30,WHW-70,Catalina
3,UK20200928-A10D,G8T1-GJ00-0301-1LUD,Pass,WHW-70//COMPLETED ON 03/30/2021,Megan Schacht-bonneau,2021-03-30,WHW-70,Catalina
4,UK20200928-A10D,G8T1-GJ00-0301-1LT7,Pass,WHW-70//COMPLETED ON 03/30/2021,Megan Schacht-bonneau,2021-03-30,WHW-70,Catalina
5,UK20200928-A10D,G8T1-GJ00-0301-1LT2,Pass,WHW-70//COMPLETED ON 03/30/2021,Megan Schacht-bonneau,2021-03-30,WHW-70,Catalina


,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,EU20200928-A5N,G8T1-GJ00-0301-207F,Pass,COMPLETED ON 03/01/2021,Megan Schacht-bonneau,2021-03-01,none,Catalina
2,EU20200928-A5N,G8T1-GJ00-0301-206N,Pass,COMPLETED ON 03/01/2021,Megan Schacht-bonneau,2021-03-01,none,Catalina
3,EU20200928-A5N,G8T1-GJ00-0301-206X,Pass,COMPLETED ON 03/01/2021,Megan Schacht-bonneau,2021-03-01,none,Catalina
4,EU20200928-A5N,G8T1-GJ00-0301-2087,Pass,COMPLETED ON 03/01/2021,Megan Schacht-bonneau,2021-03-01,none,Catalina
5,EU20200928-A5N,G8T1-GJ00-0301-207G,Pass,COMPLETED ON 03/01/2021,Megan Schacht-bonneau,2021-03-01,none,Catalina


,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,UK20201207-A19N,G8T1-GJ00-0311-CJT2,Pass,WHW-70 (SERIAL STICKER IN WRONG SPOT)//COMPLET...,Megan Schacht-bonneau,2021-03-08,WHW-70,Catalina
2,UK20201207-A19N,G8T1-GJ00-0311-CJST,Pass,WHW-70//COMPLETED ON 03/08/2021,Megan Schacht-bonneau,2021-03-08,WHW-70,Catalina
3,UK20201207-A19N,G8T1-GJ00-0311-CJSU,Pass,WHW-70//COMPLETED ON 03/08/2021,Megan Schacht-bonneau,2021-03-08,WHW-70,Catalina
4,UK20201207-A19N,G8T1-GJ00-0311-CJSV,Pass,WHW-70//COMPLETED ON 03/08/2021,Megan Schacht-bonneau,2021-03-08,WHW-70,Catalina
5,UK20201207-A19N,G8T1-GJ00-0311-CJT1,Pass,WHW-70//COMPLETED ON 03/08/2021,Megan Schacht-bonneau,2021-03-08,WHW-70,Catalina


,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,NA20210222-A1D,G8T1-GH00-1046-11RQ,Pass,COMPLETED ON 03/23/2021,Nikky Voeuth,2021-03-23,none,Catalina
2,NA20210222-A1D,G8T1-GH00-1046-11S3,Pass,COMPLETED ON 03/23/2021,Nikky Voeuth,2021-03-23,none,Catalina
3,NA20210222-A1D,G8T1-GH00-1046-11T7,Pass,COMPLETED ON 03/23/2021,Nikky Voeuth,2021-03-23,none,Catalina
4,NA20210222-A1D,G8T1-GH00-1046-11SL,Pass,COMPLETED ON 03/23/2021,Nikky Voeuth,2021-03-23,none,Catalina
5,NA20210222-A1D,G8T1-GH00-1046-11SA,Pass,COMPLETED ON 03/23/2021,Nikky Voeuth,2021-03-23,none,Catalina


,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,UK20201116-A10D,G8T1-GJ00-0311-4KDB,Pass,COMPLETED ON 03/15/2021,Megan Schacht-bonneau,2021-03-15,none,Catalina
2,UK20201116-A10D,G8T1-GJ00-0311-4GG8,Pass,COMPLETED ON 03/15/2021,Megan Schacht-bonneau,2021-03-15,none,Catalina
3,UK20201116-A10D,G8T1-GJ00-0311-4J7S,Pass,COMPLETED ON 03/15/2021,Megan Schacht-bonneau,2021-03-15,none,Catalina
4,UK20201116-A10D,G8T1-GJ00-0311-4J9K,Pass,COMPLETED ON 03/15/2021,Megan Schacht-bonneau,2021-03-15,none,Catalina
5,UK20201116-A10D,G8T1-GJ00-0311-4J75,Pass,COMPLETED ON 03/15/2021,Megan Schacht-bonneau,2021-03-15,none,Catalina


' Still need to deal with Jan, Feb, Jun, Jul'

In [14]:
# Insert Mar/Apr
results = [res1, res2, res3, res4, res5, res6, res7, res8]
for r in results:
    insertTable(r)

success!
success!
success!
success!
success!
success!
success!
success!


In [27]:
# Finished May Files
mayFiles = [f for f in allFiles if 'May' in f and 'ipynb' not in f]
mayTables = []
for path in mayFiles:
    mayTables.append(readPath(path, 'Catalina'))
for p in mayTables:
    display(p.head())
    
for r in mayTables:
    insertTable(r)

ValueError: File is not a recognized excel file

In [37]:
# Finished January
janFiles = [f for f in allFiles if 'Jan' in f and 'ipynb' not in f]
display(janFiles)
janTables = []
for path in janFiles:
    janTables.append(readPath(path, 'Catalina'))
for p in janTables:
    display(p.head())
# for r in janTables:
#     insertTable(r)

['January 2021/Catalina QC Test 01.03 01.09 2021 Restored January 08, 2021 at 08.06.49.108 AM PST.xlsx',
 'January 2021/Catalina QC Test 01.10 01.16 2021.xlsx',
 'January 2021/Catalina QC Test 01.24 01.30 2021.xlsx',
 'January 2021/Catalina QC Test 01.17 01.23 2021.xlsx']

,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,NA20201116-A5D,G8T1-GH00-0402-EBB0,Pass,Completed QC test 01/04/2021,Nikky Voeuth,2021-01-04,none,Catalina
2,NA20201116-A5D,G8T1-GH00-0402-ES35,Pass,Completed QC test 01/04/2021,Nikky Voeuth,2021-01-04,none,Catalina
3,NA20201116-A5D,G8T1-GH00-0402-EB3S,Pass,Completed QC test 01/04/2021,Nikky Voeuth,2021-01-04,none,Catalina
4,NA20201116-A5D,G8T1-GH00-0402-ERSH,Pass,Completed QC test 01/04/2021,Nikky Voeuth,2021-01-04,none,Catalina
5,NA20201116-A5D,G8T1-GH00-0402-ERS4,Pass,Completed QC test 01/04/2021,Nikky Voeuth,2021-01-04,none,Catalina


,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,NA20210104-A1N,G8T1-GH00-0453-5WEL,Pass,COMPLETED 01/11/2021,Megan Schacht-bonneau,2021-01-11,none,Catalina
2,NA20210104-A1N,G8T1-GH00-0453-5WE3,Pass,COMPLETED 01/11/2021,Megan Schacht-bonneau,2021-01-11,none,Catalina
3,NA20210104-A1N,G8T1-GH00-0453-5WEK,Pass,COMPLETED 01/11/2021,Megan Schacht-bonneau,2021-01-11,none,Catalina
4,NA20210104-A1N,G8T1-GH00-0453-5WEG,Pass,COMPLETED 01/11/2021,Megan Schacht-bonneau,2021-01-11,none,Catalina
5,NA20210104-A1N,G8T1-GH00-0453-5WEC,Pass,COMPLETED 01/11/2021,Megan Schacht-bonneau,2021-01-11,none,Catalina


,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,NA20201026-A13?,G8T1-GH00-0402-3DWJ,Pass,WHW-47 (shift in diff language) COMPLETED 1/25/21,Nikky Voeuth,2021-01-25,WHW-47,Catalina
2,NA20201026-A13?,G8T1-GH00-0402-3DVS,Pass,WHW-47 (shift in diff language) COMPLETED 1/25/21,Nikky Voeuth,2021-01-25,WHW-47,Catalina
3,NA20201026-A13?,G8T1-GH00-0402-3DVX,Pass,WHW-47 (shift in diff language) COMPLETED 1/25/21,Nikky Voeuth,2021-01-25,WHW-47,Catalina
4,NA20201026-A13?,G8T1-GH00-0402-3DW0,Pass,WHW-47 (shift in diff language) COMPLETED 1/25/21,Nikky Voeuth,2021-01-25,WHW-47,Catalina
5,NA20201026-A13?,G8T1-GH00-0402-3EA0,Pass,WHW-47 (shift in diff language) COMPLETED 1/25/21,Nikky Voeuth,2021-01-25,WHW-47,Catalina


,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,NA20210111-A10N,G8T1-GH00-0482-6U6E,Pass,COMPLETED 01/20/2021,Megan Schacht-bonneau,2021-01-20,none,Catalina
2,NA20210111-A10N,G8T1-GH00-0482-6TXG,Pass,COMPLETED 01/20/2021,Megan Schacht-bonneau,2021-01-20,none,Catalina
3,NA20210111-A10N,G8T1-GH00-0482-6U6A,Pass,COMPLETED 01/20/2021,Megan Schacht-bonneau,2021-01-20,none,Catalina
4,NA20210111-A10N,G8T1-GH00-0482-6U79,Pass,COMPLETED 01/20/2021,Megan Schacht-bonneau,2021-01-20,none,Catalina
5,NA20210111-A10N,G8T1-GH00-0482-6U6G,Pass,COMPLETED 01/20/2021,Megan Schacht-bonneau,2021-01-20,none,Catalina


In [ ]:
# Read in Mini July Data
